# Requiment 1

## Library

In [269]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import hashlib
import random
from sklearn.metrics.pairwise import cosine_similarity

## Data

In [2]:
def read_file(filename):
      with open(filename, 'r') as file:
          text = file.readlines()
      df = pd.DataFrame(text, columns=["Text"])
      return df

## LSH

In [276]:
class InMemoryMinHashLSH:

    def __init__(self, documents):
      self.documents = documents
    pass

    def convert_to_shingling(self, text, k):
      shingles = set()
      words = text.split(" ")
      for i in range(len(words) - k + 1):
          tmp = " ".join(words[i:i+k])
          shingles.add(tmp)
      return shingles

    def convert_to_boolean_vector(self, df, text):
      union_shingles = set()  # Initialize as a set
      for _, row in df.iterrows():
          union_shingles |= set(row['Shingles'])  # Use set union operation
      # Convert the set to a numpy array for efficient membership checking
      union = np.array(list(union_shingles))
      result = []
      for shingle in union:
          if shingle in text:
              result.append(1)
          else:
              result.append(0)
      return result


    def shingling(self,df):
      shingles_list = []
      for index, row in df.iterrows():
          text = row['Text'].replace(',',"").replace(';',"").replace('.',"").replace('\'',"")
          shingles = self.convert_to_shingling(text, k=10)  # Adjust k value as needed
          shingles_list.append(shingles)
      shingles_df = pd.DataFrame({'Shingles': shingles_list})

      boolean_list = []
      for index, row in df.iterrows():
          text = row['Text'].replace(',',"").replace(';',"").replace('.',"").replace('\'',"")
          boolean_vector = self.convert_to_boolean_vector(shingles_df,text)  # Adjust k value as needed
          boolean_list.append(boolean_vector)
      boolean_vector_df = pd.DataFrame({'vector': boolean_list})
      self.boolean_vector_df = boolean_vector_df
      pass

    def convert_to_signature(self,text):
      vector = np.array(text)
      arr = self.generate_arrHash(len(vector))
      result=[]
      for i in range(1,50):
        random.shuffle(arr)
        for j in range(1,len(arr)):
          if vector[arr.index(j)]==1:
            result.append(arr.index(j))
            break
      return np.array(result)

    def minhashing(self,df):
      signature_list=[]
      for index, row in df.iterrows():
        text = row['vector']
        minhash = self.convert_to_signature(text)
        signature_list.append(minhash)
      signature_df = pd.DataFrame({'signature': signature_list})

      return signature_df


      ###### Test Function
    def calculate_similarity(self,vector1, vector2):
      vector1 = np.array(vector1).reshape(1, -1)
      vector2 = np.array(vector2).reshape(1, -1)
      return cosine_similarity(vector1, vector2)[0][0]

      ############ End Test Function

    def generate_arrHash(self,n):
      arr = [i for i in range(1, n+1)]
      return arr

    def locality_sensity_hashing(self):
        # Your locality sensitive hashing implementation here
        pass

    def run(self):
      self.shingling(self.documents)
      self.minhashing(self.boolean_vector_df)
      return


    def approxNearestNeighbors(self, key, n):
        pass


In [277]:
data = read_file("/content/WebOfScience-5736.txt")
lsh = InMemoryMinHashLSH(data)
lsh.run()

     Row1  Row2  Similarity
0       0     1         0.0
1       0     2         0.0
2       0     3         0.0
3       0     4         0.0
4       0     5         0.0
..    ...   ...         ...
166    15    17         0.0
167    15    18         0.0
168    16    17         0.0
169    16    18         0.0
170    17    18         0.0

[171 rows x 3 columns]
